In [ ]:
!pip install huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -q einops
!pip install numpy==1.24
!pip install sentencepiece==0.1.98
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [ ]:
model_name = "PY007/TinyLlama-1.1B-Chat-v0.3"
model = AutoModelForCausalLM.from_pretrained(
model_name,
trust_remote_code=True,
torch_dtype=torch.bfloat16,
device_map= 'auto',
offload_folder='offload'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git

Cloning into 'llama.cpp'...
remote: Enumerating objects: 17492, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 17492 (delta 62), reused 99 (delta 51), pack-reused 17370
Receiving objects: 100% (17492/17492), 20.74 MiB | 25.56 MiB/s, done.
Resolving deltas: 100% (12124/12124), done.


In [ ]:
%cd llama.cpp

/content/llama.cpp


In [ ]:
model.save_pretrained('./models/')

In [ ]:
import os
import requests

def download_file_from_huggingface(model_name, filename, save_path):
  url = f"https://huggingface.co/{model_name}/resolve/main/{filename}"
  r = requests.get(url)
  if r.status_code != 200:

    print(f"Failed to download {filename}.HTTP Status Code: {r.status_code}")
    return False
  with open(os.path.join(save_path, filename), "wb") as f:
    f.write(r.content)
  return True

In [ ]:
def main():
  files_to_download = [
      "tokenizer_config.json",
      "tokenizer.model",
      "tokenizer.json",
      "special_tokens_map.json",
      "added_tokens.json"
      ]
  # Create the directory if it doesn't exist
  save_path="./models"
  if not os.path.exists(save_path):
    os.makedirs(save_path)

  # Download the files
  for filename in files_to_download:
      success = download_file_from_huggingface(model_name, filename, save_path)
      if success:
        print(f"Successfully downloaded {filename}")
      else:
        print(f"Failed to download {filename}")

if __name__ == "__main__":
    main()

Successfully downloaded tokenizer_config.json
Successfully downloaded tokenizer.model
Successfully downloaded tokenizer.json
Successfully downloaded special_tokens_map.json
Successfully downloaded added_tokens.json


In [ ]:
!apt update -y
!apt install build-essential git cmake libopenblas-dev libeigen3-dev -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [665 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,676 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,391 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,060 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 https://p

In [ ]:
!make LLAMA_OPENBLAS=1

I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENBLAS -I/usr/include/x86_64-linux-gnu/openblas-pthread/  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native  -Wdouble-promotion 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENBLAS -I/usr/include/x86_64-linux-gnu/openblas-pthread/  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi
I NVCCFLAGS:  
I LDFLAGS:   -L/usr/lib/x86_64-linux-gnu/openblas-pthread/ -lopenblas 
I CC:        cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
I CXX:       g++ (Ubu

In [ ]:
!python3 -m pip install -r requirements.txt

In [ ]:
!python convert.py models/

Loading model file models/model.safetensors
params = Params(n_vocab=32003, n_embd=2048, n_layer=22, n_ctx=2048, n_ff=5632, n_head=32, n_head_kv=4, n_experts=None, n_experts_used=None, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=10000.0, f_rope_scale=None, n_orig_ctx=None, rope_finetuned=None, ftype=None, path_model=PosixPath('models'))
Found vocab files: {'tokenizer.model': PosixPath('models/tokenizer.model'), 'vocab.json': None, 'tokenizer.json': PosixPath('models/tokenizer.json')}
Loading vocab file 'models/tokenizer.model', type 'spm'
Vocab info: <SentencePieceVocab with 32000 base tokens and 3 added tokens>
Special vocab info: <SpecialVocab with 0 merges, special tokens {'bos': 1, 'eos': 2, 'unk': 0, 'pad': 32000}, add special tokens unset>
Permuting layer 0
Permuting layer 1
Permuting layer 2
Permuting layer 3
Permuting layer 4
Permuting layer 5
Permuting layer 6
Permuting layer 7
Permuting layer 8
Permuting layer 9
Permuting layer 10
Permuting layer 11
Permuting la

In [ ]:
# Split the string by the '/' character
parts = model_name.split('/')

# Get the second part (index 1 because Python is 0-based)
model_name_pure = parts [1]

quant_type = "Q4_K"
quantized_model = f'models/{model_name_pure}.{quant_type}.gguf'

print(f'Preparing {quantized_model} with {quant_type} quantization.')


Preparing models/TinyLlama-1.1B-Chat-v0.3.Q4_K.gguf with Q4_K quantization.


In [ ]:
import subprocess
# Build the command as a list of arguments
command = ["./quantize", "models/ggml-model-f16.gguf", quantized_model, quant_type]
# Execute the command
subprocess.run(command)

CompletedProcess(args=['./quantize', 'models/ggml-model-f16.gguf', 'models/TinyLlama-1.1B-Chat-v0.3.Q4_K.gguf', 'Q4_K'], returncode=0)

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
quant_name = model_name.split('/') [-1]+"-GGUF" # Splits by '/' and takes the

In [ ]:
quant_name

'TinyLlama-1.1B-Chat-v0.3-GGUF'

In [ ]:
"Ayaansh/TinyLlama-1.1B-Chat-v0.3-GGUF"

In [ ]:
# Specify the repository where you want to upload the files
repo_id = "Ayaansh/" + quant_name
base_path = "./models"
# Array of local file paths you want to upload
local_file_paths = [
        base_path + "/special_tokens_map.json",
        base_path + "/tokenizer_config.json",
        base_path + "/tokenizer.json",
        base_path + "/tokenizer.model",
        base_path + "/ggml-vocab-llama.gguf",
        base_path + "/tokenizer.json",
        base_path + "/" + f'{model_name_pure}.{quant_type}.gguf',
        ]


In [ ]:
# Loop through each file and upload it
for local_file_path in local_file_paths:
  # Extract the file name from the local file path
  file_name = local_file_path.split("/")[-1]
  print(file_name)
  # Specify the path where you want the file to be uploaded in the repository
  path_in_repo = file_name # Using file_name directly, adjust as needed
  #Upload the file
  api.upload_file(
                  path_or_fileobj=local_file_path,
                  path_in_repo=path_in_repo,
                  repo_id=repo_id,
                  repo_type="model",
  )
  print (f"Uploaded {file_name} to {repo_id}")